In [15]:
!pip install -r requirements.txt
!pip install joblib

    100% |████████████████████████████████| 286kB 602kB/s ta 0:00:01


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [5]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/home/phe0/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/phe0/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
from sklearn.model_selection import StratifiedKFold

In [7]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=np.random.seed(7))
cvscores = []

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [9]:
def create_model(init='uniform', optimizer='adam'):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = init, activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 10, kernel_initializer = init, activation = 'relu'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units = 1, kernel_initializer = init, activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [10]:
model = KerasClassifier(build_fn = create_model)

In [11]:
batch_size = [5, 10, 20, 40, 60]
epochs = [10, 20, 30]
init = ['uniform', 'normal', 'glorot_uniform']
optimizer = ['adam', 'rmsprop']
param_grid = dict(batch_size = batch_size, epochs = epochs, init = init, optimizer = optimizer)

In [12]:
#for train, test in kfold.split(X, y):

grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs = -1, cv = 10)
grid_result = grid.fit(X, y)
#grid

/home/phe0/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
10000/10000 [==============================] - 2s 170us/step - loss: 0.5504 - acc: 0.7958
Epoch 2/10
10000/10000 [==============================] - 2s 219us/step - loss: 0.5109 - acc: 0.7963
Epoch 3/10
10000/10000 [==============================] - 2s 184us/step - loss: 0.5037 - acc: 0.7963
Epoch 4/10
10000/10000 [==============================] - 2s 152us/step - loss: 0.5032 - acc: 0.7963
Epoch 5/10
10000/10000 [==============================] - 1s 128us/step - loss: 0.5040 - acc: 0.7963 0s - loss: 0.5
Epoch 6/10
10000/10000 [==============================] - 1s 139us/step - loss: 0.5020 - acc: 0.7963
Epoch 7/10
10000/10000 [==============================] - 1s 121us/step - loss: 0.5013 - acc: 0.7963
Epoch 8/10
10000/10000 [===================

In [16]:
from joblib import dump, load
dump(grid_result, 'grid_result.joblib') 

['grid_result.joblib']

In [17]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.796300 using {'batch_size': 5, 'epochs': 10, 'init': 'uniform', 'optimizer': 'adam'}
0.796300 (0.010479) with: {'batch_size': 5, 'epochs': 10, 'init': 'uniform', 'optimizer': 'adam'}
0.796300 (0.010479) with: {'batch_size': 5, 'epochs': 10, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.796300 (0.010479) with: {'batch_size': 5, 'epochs': 10, 'init': 'normal', 'optimizer': 'adam'}
0.796300 (0.010479) with: {'batch_size': 5, 'epochs': 10, 'init': 'normal', 'optimizer': 'rmsprop'}
0.679500 (0.235973) with: {'batch_size': 5, 'epochs': 10, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.796100 (0.010728) with: {'batch_size': 5, 'epochs': 10, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.796300 (0.010479) with: {'batch_size': 5, 'epochs': 20, 'init': 'uniform', 'optimizer': 'adam'}
0.796200 (0.010553) with: {'batch_size': 5, 'epochs': 20, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.796300 (0.010479) with: {'batch_size': 5, 'epochs': 20, 'init': 'normal', 'optimizer': 'adam'}
0.